* https://github.com/JuliaMath/Combinatorics.jl/blob/master/src/permutations.jl#L47
* https://discourse.julialang.org/t/is-there-a-function-behaving-the-same-as-next-permutation-does-in-c/63451/13

In [1]:
module O

"""
zero allocation version of AquaIndigo's code
"""
function next_perm_old!(v::AbstractVector)
    length(v) ≤ 1 && return false    
    k = findlast(isless(v[i], v[i+1]) for i in firstindex(v):lastindex(v)-1)
    isnothing(k) && (reverse!(v); return false)
    i = k + firstindex(v) - 1    
    j = findlast(isless(v[i], v[j]) for j in i+1:lastindex(v)) + i
    v[i], v[j] = v[j], v[i]
    reverse!(v, i + 1)
    return true
end

"""
    next_perm!(v::AbstractVector)

changes `v` into the next permutation of `v` under the lexicographic order and returns `false` if it is the last permutation and `true` otherwise.

This is a variant of `Combinatorics.nextpermutation(m, t, state)`.

Examples

```
julia> v = [1, 2, 3]; println(v); while O.next_perm!(v) println(v) end
[1, 2, 3]
[1, 3, 2]
[2, 1, 3]
[2, 3, 1]
[3, 1, 2]
[3, 2, 1]
```

```
julia> v = [1, 2, 2]; println(v); while O.next_perm!(v) println(v) end
[1, 2, 2]
[2, 1, 2]
[2, 2, 1]
```
"""
function next_perm!(v::AbstractVector)
    length(v) ≤ 1 && return false
    i = lastindex(v) - 1
    @inbounds while i ≥ firstindex(v) && !isless(v[i], v[i+1]) i -= 1 end
    i < firstindex(v) && (reverse!(v); return false)
    j = lastindex(v)
    @inbounds while j > i && !isless(v[i], v[j]) j -= 1 end
    @inbounds v[i], v[j] = v[j], v[i]
    reverse!(v, i + 1)
    return true
end

function collect_perm!(v::AbstractVector)
    a = [copy(v)]
    while next_perm!(v) push!(a, copy(v)) end
    a
end

"""
    Perm(v::AbstractVector)

constructs the lexicographic order iterator of the all permutations of `v`.

Examples

```
julia> for s in O.Perm([1, 2, 3]) println(s) end
[1, 2, 3]
[1, 3, 2]
[2, 1, 3]
[2, 3, 1]
[3, 1, 2]
[3, 2, 1]
```

```
julia> for s in O.Perm([1, 2, 2]) println(s) end
[1, 2, 2]
[2, 1, 2]
[2, 2, 1]
```
"""
struct Perm{T<:AbstractVector}
    v::T
    Perm(v::T) where T<: AbstractVector = new{T}(sort(v))
end

function Base.length(p::Perm)
    v = p.v
    N = factorial(length(v))
    i = firstindex(v)
    for j in eachindex(v)
        if isless(v[i], v[j])
            N = N ÷ factorial(j-i)
            i = j
        end
    end
    N = N ÷ factorial(lastindex(v)-i+1)
    N
end

Base.eltype(p::Perm{T}) where T<:AbstractVector = T

function Base.iterate(p::Perm)
    s = copy(p.v)
    t = copy(s)
    next_perm!(t) || return (s, nothing)
    u = copy(t)
    next_perm!(u) || return (s, (t, nothing))
    return (s, (t, u))
end

function Base.iterate(p::Perm, state)
    isnothing(state) && return nothing
    s, t = state
    isnothing(t) && return (s, nothing)
    u = copy(t)
    next_perm!(u) || return (s, (t, nothing))
    return (s, (t, u))
end

#function Base.iterate(p::Perm, s = copy(p.v))
#    isnothing(s) && return nothing
#    next_perm!(s) || return (s, nothing)
#    return (copy(s), s)
#end

end

Main.O

In [2]:
@doc O.next_perm!

```
next_perm!(v::AbstractVector)
```

changes `v` into the next permutation of `v` under the lexicographic order and returns `false` if it is the last permutation and `true` otherwise.

This is a variant of `Combinatorics.nextpermutation(m, t, state)`.

Examples

```
julia> v = [1, 2, 3]; println(v); while O.next_perm!(v) println(v) end
[1, 2, 3]
[1, 3, 2]
[2, 1, 3]
[2, 3, 1]
[3, 1, 2]
[3, 2, 1]
```

```
julia> v = [1, 2, 2]; println(v); while O.next_perm!(v) println(v) end
[1, 2, 2]
[2, 1, 2]
[2, 2, 1]
```


In [3]:
@doc O.Perm

```
Perm(v::AbstractVector)
```

constructs the lexicographic order iterator of the all permutations of `v`.

Examples

```
julia> for s in O.Perm([1, 2, 3]) println(s) end
[1, 2, 3]
[1, 3, 2]
[2, 1, 3]
[2, 3, 1]
[3, 1, 2]
[3, 2, 1]
```

```
julia> for s in O.Perm([1, 2, 2]) println(s) end
[1, 2, 2]
[2, 1, 2]
[2, 2, 1]
```


In [4]:
v = [1, 2, 3]; println(v); while O.next_perm!(v) println(v) end

[1, 2, 3]
[1, 3, 2]
[2, 1, 3]
[2, 3, 1]
[3, 1, 2]
[3, 2, 1]


In [5]:
v = [1, 2, 2]; println(v); while O.next_perm!(v) println(v) end

[1, 2, 2]
[2, 1, 2]
[2, 2, 1]


In [6]:
for s in O.Perm([1, 2, 3]) println(s) end

[1, 2, 3]
[1, 3, 2]
[2, 1, 3]
[2, 3, 1]
[3, 1, 2]
[3, 2, 1]


In [7]:
for s in O.Perm([1, 2, 2]) println(s) end

[1, 2, 2]
[2, 1, 2]
[2, 2, 1]


In [8]:
v = sort([NaN, NaN, 2, 1])
@show v
while O.next_perm!(v)
    @show v
end

v = [1.0, 2.0, NaN, NaN]
v = [1.0, NaN, 2.0, NaN]
v = [1.0, NaN, NaN, 2.0]
v = [2.0, 1.0, NaN, NaN]
v = [2.0, NaN, 1.0, NaN]
v = [2.0, NaN, NaN, 1.0]
v = [NaN, 1.0, 2.0, NaN]
v = [NaN, 1.0, NaN, 2.0]
v = [NaN, 2.0, 1.0, NaN]
v = [NaN, 2.0, NaN, 1.0]
v = [NaN, NaN, 1.0, 2.0]
v = [NaN, NaN, 2.0, 1.0]


In [9]:
p = O.Perm([NaN, NaN, 2, 1])
for s in p
    @show s
end

s = [1.0, 2.0, NaN, NaN]
s = [1.0, NaN, 2.0, NaN]
s = [1.0, NaN, NaN, 2.0]
s = [2.0, 1.0, NaN, NaN]
s = [2.0, NaN, 1.0, NaN]
s = [2.0, NaN, NaN, 1.0]
s = [NaN, 1.0, 2.0, NaN]
s = [NaN, 1.0, NaN, 2.0]
s = [NaN, 2.0, 1.0, NaN]
s = [NaN, 2.0, NaN, 1.0]
s = [NaN, NaN, 1.0, 2.0]
s = [NaN, NaN, 2.0, 1.0]


In [10]:
using BenchmarkTools

In [11]:
p = O.Perm([NaN, NaN, 2, 1])
_, state = iterate(p)

([1.0, 2.0, NaN, NaN], ([1.0, NaN, 2.0, NaN], [1.0, NaN, NaN, 2.0]))

In [12]:
iterate(p, state)

([1.0, NaN, 2.0, NaN], ([1.0, NaN, NaN, 2.0], [2.0, 1.0, NaN, NaN]))

In [13]:
p = O.Perm([NaN, NaN, 2, 1])
@btime collect($p)

  526.316 ns (25 allocations: 1.92 KiB)


12-element Vector{Vector{Float64}}:
 [1.0, 2.0, NaN, NaN]
 [1.0, NaN, 2.0, NaN]
 [1.0, NaN, NaN, 2.0]
 [2.0, 1.0, NaN, NaN]
 [2.0, NaN, 1.0, NaN]
 [2.0, NaN, NaN, 1.0]
 [NaN, 1.0, 2.0, NaN]
 [NaN, 1.0, NaN, 2.0]
 [NaN, 2.0, 1.0, NaN]
 [NaN, 2.0, NaN, 1.0]
 [NaN, NaN, 1.0, 2.0]
 [NaN, NaN, 2.0, 1.0]

In [14]:
v = sort([NaN, NaN, 2, 1])
@btime O.collect_perm!($v)

  482.653 ns (15 allocations: 1.81 KiB)


12-element Vector{Vector{Float64}}:
 [1.0, 2.0, NaN, NaN]
 [1.0, NaN, 2.0, NaN]
 [1.0, NaN, NaN, 2.0]
 [2.0, 1.0, NaN, NaN]
 [2.0, NaN, 1.0, NaN]
 [2.0, NaN, NaN, 1.0]
 [NaN, 1.0, 2.0, NaN]
 [NaN, 1.0, NaN, 2.0]
 [NaN, 2.0, 1.0, NaN]
 [NaN, 2.0, NaN, 1.0]
 [NaN, NaN, 1.0, 2.0]
 [NaN, NaN, 2.0, 1.0]

In [15]:
test_next_perm!(v) = (c = 1; while O.next_perm!(v) c += 1 end; c)
@show s = [1, 1, 2, 2, 2, 3, 4, 5, 6, 6]
@btime test_next_perm!($s)

s = [1, 1, 2, 2, 2, 3, 4, 5, 6, 6] = [1, 1, 2, 2, 2, 3, 4, 5, 6, 6]
  1.296 ms (0 allocations: 0 bytes)


151200

In [16]:
test_next_perm_old!(v) = (c = 1; while O.next_perm_old!(v) c += 1 end; c)
@show s = [1, 1, 2, 2, 2, 3, 4, 5, 6, 6]
@btime test_next_perm_old!($s)

s = [1, 1, 2, 2, 2, 3, 4, 5, 6, 6] = [1, 1, 2, 2, 2, 3, 4, 5, 6, 6]
  5.484 ms (0 allocations: 0 bytes)


151200

In [17]:
c = @btime collect($(O.Perm(s)))
length(c)

  6.450 ms (302402 allocations: 28.84 MiB)


151200

In [18]:
@code_warntype O.next_perm!(s)

MethodInstance for Main.O.next_perm!(::Vector{Int64})
  from next_perm!(v::AbstractVector) in Main.O at In[1]:43
Arguments
  #self#::Core.Const(Main.O.next_perm!)
  v::Vector{Int64}
Locals
  @_3::Int64
  val@_4::Tuple{Int64, Int64}
  val@_5::Nothing
  val@_6::Nothing
  j::Int64
  i::Int64
  @_9::Bool
  @_10::Bool
Body::Bool
1 ──       Core.NewvarNode(:(@_3))
│          Core.NewvarNode(:(val@_4))
│          Core.NewvarNode(:(val@_5))
│          Core.NewvarNode(:(val@_6))
│          Core.NewvarNode(:(j))
│          Core.NewvarNode(:(i))
│    %7  = Main.O.length(v)::Int64
│    %8  = (%7 ≤ 1)::Bool
└───       goto #3 if not %8
2 ──       return false
3 ── %11 = Main.O.lastindex(v)::Int64
│          (i = %11 - 1)
└───       $(Expr(:inbounds, true))
4 ┄─ %14 = i::Int64
│    %15 = Main.O.firstindex(v)::Core.Const(1)
│    %16 = (%14 ≥ %15)::Bool
└───       goto #6 if not %16
5 ── %18 = Base.getindex(v, i)::Int64
│    %19 = (i + 1)::Int64
│    %20 = Base.getindex(v, %19)::Int64
│    %21 = Main.

In [ ]:
using OffsetArrays
@show t = OffsetArray([1, 1, 2, 2, 2, 3, 4, 5, 6, 6], -5:4)
@btime collect($(O.Perm(t)))

In [ ]:
@btime O.collect_perm!($(sort(t)))

In [ ]:
@code_warntype O.next_perm!(t)